# 🧠 Selección de Características + Modelado Predictivo
**Nombre:** Gabriela Ocampo  
**Asignatura:** Programación Orientada a Objetos  

Este notebook aplica un reto adicional sobre el dataset Titanic, enfocándose en la **selección de características** para mejorar el rendimiento de los modelos de clasificación y regresión.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_classif, f_regression


## 1️⃣ Carga y preparación del dataset

In [2]:
# Cargar dataset Titanic desde seaborn
df = load_dataset('titanic')
# Eliminar columnas no útiles y filas con nulos
df = df.drop(columns=['deck', 'embark_town', 'alive', 'class', 'who'])
df = df.dropna(subset=['age', 'embarked', 'fare', 'sex'])
# Codificar categóricas
df = pd.get_dummies(df, columns=['sex', 'embarked'], drop_first=True)
# Normalizar numéricas
df[['age', 'fare']] = (df[['age', 'fare']] - df[['age', 'fare']].mean()) / df[['age', 'fare']].std()
df.head()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_male,embarked_Q,embarked_S
0,0,3,-0.527298,1,0,-0.516017,True,False,True,False,True
1,1,1,0.576688,1,0,0.693558,False,False,False,False,False
2,1,3,-0.251301,0,0,-0.503267,False,True,False,False,True
3,1,1,0.369691,1,0,0.350080,False,False,False,False,True
4,0,3,0.369691,0,0,-0.500905,True,True,True,False,True


## 2️⃣ Selección de Características - Clasificación

In [3]:
# Separar datos para clasificación
X_cls = df.drop(columns=['survived'])
y_cls = df['survived']
# Aplicar SelectKBest
selector_cls = SelectKBest(score_func=f_classif, k=5)
X_cls_selected = selector_cls.fit_transform(X_cls, y_cls)
selected_cols_cls = X_cls.columns[selector_cls.get_support()]

In [4]:
# Clasificación con características seleccionadas
X_train, X_test, y_train, y_test = train_test_split(X_cls[selected_cols_cls], y_cls, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("🔍 Clasificación (SelectKBest):")
print(classification_report(y_test, y_pred))

🔍 Clasificación (SelectKBest):
              precision    recall  f1-score   support

           0       0.77      0.85      0.81        80
           1       0.78      0.68      0.73        63

    accuracy                           0.78       143
   macro avg       0.78      0.77      0.77       143
weighted avg       0.78      0.78      0.77       143



## 3️⃣ Selección de Características - Regresión

In [5]:
# Separar datos para regresión
X_reg = df.drop(columns=['fare'])
y_reg = df['fare']
# Aplicar SelectKBest
selector_reg = SelectKBest(score_func=f_regression, k=5)
X_reg_selected = selector_reg.fit_transform(X_reg, y_reg)
selected_cols_reg = X_reg.columns[selector_reg.get_support()]

In [6]:
# Regresión con características seleccionadas
Xr_train, Xr_test, yr_train, yr_test = train_test_split(df[selected_cols_reg], y_reg, test_size=0.2, random_state=42)
reg = LinearRegression()
reg.fit(Xr_train, yr_train)
yr_pred = reg.predict(Xr_test)
print("📈 Regresión (SelectKBest):")
print("MSE:", mean_squared_error(yr_test, yr_pred))
print("R2:", r2_score(yr_test, yr_pred))

📈 Regresión (SelectKBest):
MSE: 1.2756588793672619
R2: 0.30922817626084853


## ✅ Conclusión
- La selección de características permite reducir dimensionalidad y mejorar el rendimiento.
- SelectKBest ayuda a identificar las variables más relevantes para cada tipo de modelo.
- Ambas tareas (clasificación y regresión) se beneficiaron del preprocesamiento y selección correcta.